In [4]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date

In [109]:
today = "2020-03-16"

In [110]:
URL = "https://web.archive.org/web/20200316221106/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"


In [111]:
page = requests.get(URL)

In [112]:
soup = BeautifulSoup(page.content, 'html.parser')

In [113]:
results = soup.find_all("tr")

In [115]:
results[1]

<tr class="even"><td colspan="1" rowspan="1">Baden-Württemberg</td><td class="center" colspan="1" rowspan="1">1.105 (3)</td><td class="center" colspan="1" rowspan="1">1.062 (3)</td><td colspan="1" rowspan="1"></td></tr>

In [116]:
bundesland = []
faelle = []
tod = []
for idx,elm in enumerate(results):
    if idx > 0:
        bundesland.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[0])))
        faelle.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()[0])
        if len(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()) > 1:
            tod.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()[1].strip("(").strip(")"))
        else:
            tod.append("0")
    else:
        pass

In [117]:
faelle

['1.105',
 '1.067',
 '300',
 '94',
 '56',
 '260',
 '342',
 '51',
 '391',
 '1.541',
 '325',
 '85',
 '140',
 '77',
 '123',
 '55',
 '6.012']

In [118]:
lat={"Baden-Württemberg":48.5, "Bayern":48.9, "Berlin":52.5, "Brandenburg":52, "Bremen":53.1, "Hamburg":53.5, 
     "Hessen":50.7, "Mecklenburg-Vorpommern":53.8, "Niedersachsen":52.57, "Nordrhein-Westfalen":51.6,"Rheinland-Pfalz":50, 
     "Saarland":49.4, "Sachsen":51.05, "Sachsen-Anhalt":52, "Schleswig-Holstein":54.2, "Thüringen":50.85, "Gesamt":51.06}
long={"Baden-Württemberg":9, "Bayern":11.4, "Berlin":13.4, "Brandenburg":13.87, "Bremen":8.8, "Hamburg":10, 
      "Hessen":9, "Mecklenburg-Vorpommern":12.5,"Niedersachsen":10, "Nordrhein-Westfalen":7.5,"Rheinland-Pfalz":7.3,
      "Saarland":6.97,"Sachsen":13.5, "Sachsen-Anhalt":11.65,"Schleswig-Holstein":9.9, "Thüringen":11.13,"Gesamt":10}


In [119]:
df=pd.DataFrame(data=[bundesland,faelle,tod]).T
    

In [120]:
df["lat"]="NaN"
df["long"]="NaN"
df["date"] = today

In [121]:
df.columns = ["bundesland","faelle","tod","lat","long","datum"]

In [122]:
df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)

In [123]:
df.datum = pd.to_datetime(df.datum)

In [124]:
df.bundesland.replace("Schleswig Holstein","Schleswig-Holstein", inplace=True)
lat2=[]
long2=[]
for elm in df.bundesland.values:
    lat2.append(lat[elm])
    long2.append(long[elm])

In [125]:
df["lat"] = lat2
df["long"] = long2

In [126]:
df

,bundesland,faelle,tod,lat,long,datum
0,Baden-Württemberg,1.105,3,48.50,9.00,2020-03-16
1,Bayern,1.067,5,48.90,11.40,2020-03-16
2,Berlin,300,0,52.50,13.40,2020-03-16
3,Brandenburg,94,0,52.00,13.87,2020-03-16
4,Bremen,56,0,53.10,8.80,2020-03-16
5,Hamburg,260,0,53.50,10.00,2020-03-16
6,Hessen,342,0,50.70,9.00,2020-03-16
7,Mecklenburg-Vorpommern,51,0,53.80,12.50,2020-03-16
8,Niedersachsen,391,0,52.57,10.00,2020-03-16
9,Nordrhein-Westfalen,1.541,5,51.60,7.50,2020-03-16


In [127]:
for idx,elm in enumerate(df.faelle):
    df.faelle[idx] = elm.replace(".","")

for idx,elm in enumerate(df.tod):
    df.tod[idx] = elm.replace(".","")

/Users/karstenyan/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/karstenyan/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 6 columns):
bundesland    17 non-null object
faelle        17 non-null object
tod           17 non-null object
lat           17 non-null float64
long          17 non-null float64
datum         17 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 896.0+ bytes


In [129]:
#df.lat = df.lat.astype("float")
#df.long = df.long.astype("float")
df.tod = df.tod.astype("int64")
df.faelle = df.faelle.astype("int64")

In [130]:
df[df["datum"]==today]

,bundesland,faelle,tod,lat,long,datum
0,Baden-Württemberg,1105,3,48.50,9.00,2020-03-16
1,Bayern,1067,5,48.90,11.40,2020-03-16
2,Berlin,300,0,52.50,13.40,2020-03-16
3,Brandenburg,94,0,52.00,13.87,2020-03-16
4,Bremen,56,0,53.10,8.80,2020-03-16
5,Hamburg,260,0,53.50,10.00,2020-03-16
6,Hessen,342,0,50.70,9.00,2020-03-16
7,Mecklenburg-Vorpommern,51,0,53.80,12.50,2020-03-16
8,Niedersachsen,391,0,52.57,10.00,2020-03-16
9,Nordrhein-Westfalen,1541,5,51.60,7.50,2020-03-16


In [131]:
df.to_csv ('old_data/brd_covid_19'+str(today)+".csv", index = False, header=True)